In [1]:
from requests import get
import glob
import random
import base64
import pandas as pd
from time import sleep
from random import randint

from PIL import Image
from io import BytesIO
from IPython.display import HTML

url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'

response = get(url)
print(response.text[:500])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle",


In [2]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

from time import time
from IPython.core.display import clear_output

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
from warnings import warn

warn("Warning Simulation")


/home/sarvagya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Warning Simulation


In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [4]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
image_urls = []
genre_seg = []
actors = []
plot = []
director = []

pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2011,2019)]

from urllib.request import urlopen
from PIL import Image
import numpy as np

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	img = Image.fromarray(image, 'RGB')
	# return the image
	image_urls.append(img)


# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
        
                #Image URL
                image = container.find('div', class_ = 'lister-item-image float-left')
                image_url = image.find('img', class_ = 'loadlate')

                image_urls.append((image_url['loadlate']))
                
                #Genre
                genre_div = container.find('div' , class_ = 'lister-item-content')
                genre_p = genre_div.find('p' , class_ = 'text-muted')
                genre_span = genre_p.find('span' , class_ = 'genre').text.strip()
                genre = genre_span.split(",")
                genre_seg.append(genre)
                
                #Actors And Directors
                actor_tag = container.find_all('p')[2]
                actor = actor_tag.find_all('a')
                act = []
                x = 0
                for i in actor:
                    if(x == 0):
                        director.append(i.text)
                    else:
                        act.append(i.text)
                    x+=1
                actors.append(act)
                
                #Plot
                plot_q = container.find_all('p')[1].get_text()
                plot.append(plot_q.strip())

Request:32; Frequency: 0.07011342030764607 requests/s


In [5]:
import pandas as pd

test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes,
                       'image_urls':image_urls,
                       'genre':genre_seg,
                       'actors':actors,
                        'director':director})
test_df.drop_duplicates('movie')
test_df.index += 1
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 1 to 1472
Data columns (total 9 columns):
movie         1472 non-null object
year          1472 non-null object
imdb          1472 non-null float64
metascore     1472 non-null int64
votes         1472 non-null int64
image_urls    1472 non-null object
genre         1472 non-null object
actors        1472 non-null object
director      1472 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 103.6+ KB
None


,movie,year,imdb,metascore,votes,image_urls,genre,actors,director
1,Harry Potter and the Deathly Hallows: Part 2,(2011),8.1,87,677148,https://m.media-amazon.com/images/M/MV5BMjIyZG...,"[Adventure, Drama, Fantasy]","[Daniel Radcliffe, Emma Watson, Rupert Grint, ...",David Yates
2,The Intouchables,(2011),8.5,57,657362,https://m.media-amazon.com/images/M/MV5BMTYxND...,"[Biography, Comedy, Drama]","[Éric Toledano, François Cluzet, Omar Sy, Anne...",Olivier Nakache
3,Thor,(2011),7.0,57,656498,https://m.media-amazon.com/images/M/MV5BOGE4Nz...,"[Action, Adventure, Fantasy]","[Chris Hemsworth, Anthony Hopkins, Natalie Por...",Kenneth Branagh
4,Captain America: The First Avenger,(2011),6.9,66,635886,https://m.media-amazon.com/images/M/MV5BMTYzOT...,"[Action, Adventure, Sci-Fi]","[Chris Evans, Hugo Weaving, Samuel L. Jackson,...",Joe Johnston
5,X: First Class,(2011),7.7,65,597588,https://m.media-amazon.com/images/M/MV5BMTg5OT...,"[Action, Adventure, Sci-Fi]","[James McAvoy, Michael Fassbender, Jennifer La...",Matthew Vaughn
6,Drive,(I) (2011),7.8,78,512685,https://m.media-amazon.com/images/M/MV5BZjY5Zj...,"[Crime, Drama]","[Ryan Gosling, Carey Mulligan, Bryan Cranston,...",Nicolas Winding Refn
7,Limitless,(I) (2011),7.4,59,484435,https://m.media-amazon.com/images/M/MV5BYmViZG...,"[Mystery, Sci-Fi, Thriller]","[Bradley Cooper, Anna Friel, Abbie Cornish, Ro...",Neil Burger
8,Rise of the Planet of the Apes,(2011),7.6,68,467113,https://m.media-amazon.com/images/M/MV5BYzE3Zm...,"[Action, Drama, Sci-Fi]","[James Franco, Andy Serkis, Freida Pinto, Kari...",Rupert Wyatt
9,Source Code,(2011),7.5,74,445442,https://m.media-amazon.com/images/M/MV5BMTY0MT...,"[Sci-Fi, Thriller]","[Jake Gyllenhaal, Michelle Monaghan, Vera Farm...",Duncan Jones
10,Pirates of the Caribbean: On Stranger Tides,(2011),6.6,45,439885,https://m.media-amazon.com/images/M/MV5BMjE5Mj...,"[Action, Adventure, Fantasy]","[Johnny Depp, Penélope Cruz, Ian McShane, Geof...",Rob Marshall


In [6]:
all_genres = []
for i in genre_seg:
    for j in i:
        j = j.strip()
        if j not in all_genres:
            all_genres.append(j)
test_df_1 = pd.DataFrame({'movie': names,
                          'year': years,
                          'votes':votes,
                          'genre':genre_seg,
                          'imdb':imdb_ratings})
for i in all_genres:
    test_df_1[i] = 0
all_genres
#test_df = test_df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],axis = 1)
#test_df_1

['Adventure',
 'Drama',
 'Fantasy',
 'Biography',
 'Comedy',
 'Action',
 'Sci-Fi',
 'Crime',
 'Mystery',
 'Thriller',
 'Romance',
 'Sport',
 'Horror',
 'Family',
 'Animation',
 'Western',
 'History',
 'Musical',
 'Music',
 'War']

In [7]:
for x in range(0 ,len(test_df_1)):
    for i in test_df_1.iloc[x]['genre']:
        i = i.strip()
        #print(i, end =" ")
        test_df_1.iloc[x, test_df_1.columns.get_loc(i)] = 1
        #print(i , test_df_1.iloc[x][i] , end=" ")
        #print(test_df_1.loc[x][i])
    #print("")
test_df_1

,movie,year,votes,genre,imdb,Adventure,Drama,Fantasy,Biography,Comedy,...,Romance,Sport,Horror,Family,Animation,Western,History,Musical,Music,War
0,Harry Potter and the Deathly Hallows: Part 2,(2011),677148,"[Adventure, Drama, Fantasy]",8.1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Intouchables,(2011),657362,"[Biography, Comedy, Drama]",8.5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Thor,(2011),656498,"[Action, Adventure, Fantasy]",7.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Captain America: The First Avenger,(2011),635886,"[Action, Adventure, Sci-Fi]",6.9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,X: First Class,(2011),597588,"[Action, Adventure, Sci-Fi]",7.7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Drive,(I) (2011),512685,"[Crime, Drama]",7.8,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Limitless,(I) (2011),484435,"[Mystery, Sci-Fi, Thriller]",7.4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Rise of the Planet of the Apes,(2011),467113,"[Action, Drama, Sci-Fi]",7.6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Source Code,(2011),445442,"[Sci-Fi, Thriller]",7.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Pirates of the Caribbean: On Stranger Tides,(2011),439885,"[Action, Adventure, Fantasy]",6.6,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
C = test_df_1['imdb'].mean()
#m= test_df_1['votes'].quantile(0.5)
m = 0
q_movies = test_df_1.copy().loc[test_df_1['votes'] >= m]
def weighted_rating(x, m=m, C=C):
    v = x['votes']
    R = x['imdb']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
C,m,q_movies.shape
q_movies

,movie,year,votes,genre,imdb,Adventure,Drama,Fantasy,Biography,Comedy,...,Romance,Sport,Horror,Family,Animation,Western,History,Musical,Music,War
0,Harry Potter and the Deathly Hallows: Part 2,(2011),677148,"[Adventure, Drama, Fantasy]",8.1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Intouchables,(2011),657362,"[Biography, Comedy, Drama]",8.5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Thor,(2011),656498,"[Action, Adventure, Fantasy]",7.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Captain America: The First Avenger,(2011),635886,"[Action, Adventure, Sci-Fi]",6.9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,X: First Class,(2011),597588,"[Action, Adventure, Sci-Fi]",7.7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Drive,(I) (2011),512685,"[Crime, Drama]",7.8,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Limitless,(I) (2011),484435,"[Mystery, Sci-Fi, Thriller]",7.4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Rise of the Planet of the Apes,(2011),467113,"[Action, Drama, Sci-Fi]",7.6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Source Code,(2011),445442,"[Sci-Fi, Thriller]",7.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Pirates of the Caribbean: On Stranger Tides,(2011),439885,"[Action, Adventure, Fantasy]",6.6,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
#q_movies = q_movies.sort_values('score', ascending=False)
#Print the top 10 movies
#q_movies = q_movies[['movie', 'imdb' , 'votes' , 'score', 'Adventure', 'Drama', 'Fantasy', 'Biography', 'Comedy', 'Action', 'Sci-Fi', 'Crime', 'Mystery', 'Thriller', 'Romance', 'Sport', 'Horror', 'Family', 'Animation', 'Western', 'History', 'Musical','Music','War']].drop_duplicates()
q_movies = q_movies.drop_duplicates(subset = 'movie')
q_movies

,movie,year,votes,genre,imdb,Adventure,Drama,Fantasy,Biography,Comedy,...,Romance,Sport,Horror,Family,Animation,Western,History,Musical,Music,War
0,Harry Potter and the Deathly Hallows: Part 2,(2011),677148,"[Adventure, Drama, Fantasy]",8.1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Intouchables,(2011),657362,"[Biography, Comedy, Drama]",8.5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Thor,(2011),656498,"[Action, Adventure, Fantasy]",7.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Captain America: The First Avenger,(2011),635886,"[Action, Adventure, Sci-Fi]",6.9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,X: First Class,(2011),597588,"[Action, Adventure, Sci-Fi]",7.7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Drive,(I) (2011),512685,"[Crime, Drama]",7.8,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Limitless,(I) (2011),484435,"[Mystery, Sci-Fi, Thriller]",7.4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Rise of the Planet of the Apes,(2011),467113,"[Action, Drama, Sci-Fi]",7.6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Source Code,(2011),445442,"[Sci-Fi, Thriller]",7.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Pirates of the Caribbean: On Stranger Tides,(2011),439885,"[Action, Adventure, Fantasy]",6.6,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
##
q_movies.shape

(368, 25)

In [11]:
for x in range(0 ,len(q_movies)):
    for i in q_movies.iloc[x]['genre']:
        i = i.strip()
        #print(i, end =" ")
        q_movies.iloc[x, q_movies.columns.get_loc(i)] = (q_movies.iloc[x , q_movies.columns.get_loc('imdb')]) 
q_movies = q_movies.reset_index(drop = True)

/home/sarvagya/.local/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
import numpy as np; import pandas as pd

df = pd.DataFrame(np.random.randint(0, 2, (3, 5)))

#df
##     0  1  2  3  4
##  0  1  1  1  0  0
##  1  0  0  1  1  1
##  2  0  1  0  1  0
X = q_movies
X = X.drop(['movie' , 'year' , 'votes' , 'genre'], axis=1)
#cosine_similarity(X)
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
# Compute the cosine similarity matrix
#cosine_sim = linear_kernel(X , X)
cosine_sim = cosine_similarity(X , X)
#cosine_sim
##  array([[ 1.        ,  0.33333333,  0.40824829],
##         [ 0.33333333,  1.        ,  0.40824829],
##         [ 0.40824829,  0.40824829,  1.        ]])

In [13]:
indices = pd.Series(q_movies.index, index=q_movies['movie']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:16]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return q_movies['movie'].iloc[movie_indices]
indices


movie
Harry Potter and the Deathly Hallows: Part 2      0
The Intouchables                                  1
Thor                                              2
Captain America: The First Avenger                3
X: First Class                                    4
Drive                                             5
Limitless                                         6
Rise of the Planet of the Apes                    7
Source Code                                       8
Pirates of the Caribbean: On Stranger Tides       9
Crazy, Stupid, Love.                             10
The Hangover Part II                             11
Mission: Impossible - Ghost Protocol             12
Warrior                                          13
Horrible Bosses                                  14
Sherlock Holmes: A Game of Shadows               15
The Help                                         16
The Girl with the Dragon Tattoo                  17
Transformers: Dark of the Moon                   18
Midnig

In [14]:
get_recommendations('Avengers: Infinity War')

9           Pirates of the Caribbean: On Stranger Tides
97                                         Man of Steel
103                                Thor: The Dark World
132                                        Ender's Game
184          Star Wars: Episode VII - The Force Awakens
230                  Batman v Superman: Dawn of Justice
232                                       Suicide Squad
234                                      Doctor Strange
247                                            Warcraft
277                                        Wonder Woman
280             Star Wars: Episode VIII - The Last Jedi
288                                      Justice League
294                                  Kong: Skull Island
297    Pirates of the Caribbean: Dead Men Tell No Tales
308                                           The Mummy
Name: movie, dtype: object

In [15]:
# qwerty = []
# for i in director:
#     if i not in qwerty:
#         qwerty.append(i)
# qwerty


In [16]:
print(q_movies.loc[q_movies['Adventure'] > 0].shape)
print(q_movies.loc[q_movies['Drama'] > 0].shape)
print(q_movies.loc[q_movies['Fantasy'] > 0].shape)
print(q_movies.loc[q_movies['Biography'] > 0].shape)
print(q_movies.loc[q_movies['Comedy'] > 0].shape)
print(q_movies.loc[q_movies['Action'] > 0].shape)
print(q_movies.loc[q_movies['Sci-Fi'] > 0].shape)
print(q_movies.loc[q_movies['Crime'] > 0].shape)
print(q_movies.loc[q_movies['Mystery'] > 0].shape)
print(q_movies.loc[q_movies['Thriller'] > 0].shape)
print(q_movies.loc[q_movies['Romance'] > 0].shape)
print(q_movies.loc[q_movies['Sport'] > 0].shape)
print(q_movies.loc[q_movies['Horror'] > 0].shape)
print(q_movies.loc[q_movies['Family'] > 0].shape)
print(q_movies.loc[q_movies['Animation'] > 0].shape)
print(q_movies.loc[q_movies['Western'] > 0].shape)
print(q_movies.loc[q_movies['History'] > 0].shape)
print(q_movies.loc[q_movies['Musical'] > 0].shape)
print(q_movies.loc[q_movies['Music'] > 0].shape)
print(q_movies.loc[q_movies['War'] > 0].shape)

(153, 25)
(152, 25)
(45, 25)
(33, 25)
(93, 25)
(184, 25)
(78, 25)
(65, 25)
(39, 25)
(64, 25)
(27, 25)
(4, 25)
(25, 25)
(12, 25)
(24, 25)
(3, 25)
(14, 25)
(4, 25)
(5, 25)
(2, 25)


In [17]:
final = pd.DataFrame()

final = final.append(q_movies.loc[q_movies['Adventure'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Drama'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Fantasy'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Biography'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Comedy'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Action'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Sci-Fi'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Crime'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Mystery'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Thriller'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Romance'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Sport'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Horror'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Family'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Animation'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Western'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['History'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Musical'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['Music'] > 0].head(1) , ignore_index = True)
final = final.append(q_movies.loc[q_movies['War'] > 0].head(1) , ignore_index = True)
final = final.drop_duplicates('movie')
final

,movie,year,votes,genre,imdb,Adventure,Drama,Fantasy,Biography,Comedy,...,Romance,Sport,Horror,Family,Animation,Western,History,Musical,Music,War
0,Harry Potter and the Deathly Hallows: Part 2,(2011),677148,"[Adventure, Drama, Fantasy]",8.1,8.1,8.1,8.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,The Intouchables,(2011),657362,"[Biography, Comedy, Drama]",8.5,0.0,8.5,0.0,8.5,8.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Thor,(2011),656498,"[Action, Adventure, Fantasy]",7.0,7.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Captain America: The First Avenger,(2011),635886,"[Action, Adventure, Sci-Fi]",6.9,6.9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Drive,(I) (2011),512685,"[Crime, Drama]",7.8,0.0,7.8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Limitless,(I) (2011),484435,"[Mystery, Sci-Fi, Thriller]",7.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,"Crazy, Stupid, Love.",(2011),431210,"[Comedy, Drama, Romance]",7.4,0.0,7.4,0.0,0.0,7.4,...,7.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,Warrior,(2011),397810,"[Drama, Sport]",8.2,0.0,8.2,0.0,0.0,0.0,...,0.0,8.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,The Cabin in the Woods,(2011),332355,"[Comedy, Horror]",7.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,Hugo,(2011),281128,"[Adventure, Drama, Family]",7.5,7.5,7.5,0.0,0.0,0.0,...,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
movies_to_be_rated = final['movie']
movies_to_be_rated

0     Harry Potter and the Deathly Hallows: Part 2
3                                 The Intouchables
5                                             Thor
6               Captain America: The First Avenger
7                                            Drive
8                                        Limitless
10                            Crazy, Stupid, Love.
11                                         Warrior
12                          The Cabin in the Woods
13                                            Hugo
14                                 Kung Fu Panda 2
15                                Django Unchained
16                                  Les Misérables
18                                   Pitch Perfect
19                                            Fury
Name: movie, dtype: object

In [20]:
ratings = []
for i in range(len(movies_to_be_rated)):
    x = int(input())
    ratings.append(x)
ratings

3
5
1
4
2
5
1
2
4
3
5
3
2
4
1


[3, 5, 1, 4, 2, 5, 1, 2, 4, 3, 5, 3, 2, 4, 1]

In [21]:
recommendations = pd.DataFrame(columns = ['movie' , 'rating' , 'count' , 'score'])

In [22]:
recommendations['movie'] = q_movies['movie']
recommendations['rating'] = 0
recommendations['count'] = 0
recommendations['score'] = 0

In [23]:
recommendations.reset_index(drop = True)

,movie,rating,count,score
0,Harry Potter and the Deathly Hallows: Part 2,0,0,0
1,The Intouchables,0,0,0
2,Thor,0,0,0
3,Captain America: The First Avenger,0,0,0
4,X: First Class,0,0,0
5,Drive,0,0,0
6,Limitless,0,0,0
7,Rise of the Planet of the Apes,0,0,0
8,Source Code,0,0,0
9,Pirates of the Caribbean: On Stranger Tides,0,0,0


In [24]:
rated_recom = []
for i in movies_to_be_rated:
    rated_recom.append(get_recommendations(i))
rated_recom

[42       The Twilight Saga: Breaking Dawn - Part 1
 55                                      Life of Pi
 85       The Twilight Saga: Breaking Dawn - Part 2
 274    Miss Peregrine's Home for Peculiar Children
 289                             The Shape of Water
 100            The Hobbit: The Desolation of Smaug
 150      The Hobbit: The Battle of the Five Armies
 2                                             Thor
 9      Pirates of the Caribbean: On Stranger Tides
 27                                            Hugo
 32                                        The Grey
 49               The Hobbit: An Unexpected Journey
 65                                Moonrise Kingdom
 71                     Snow White and the Huntsman
 97                                    Man of Steel
 Name: movie, dtype: object,
 199                                      The Big Short
 314                                           I, Tonya
 320                                The Disaster Artist
 112                   

In [25]:
i = 0
for j in rated_recom:
    for k in j:
        recommendations.loc[recommendations['movie'] == k, 'rating'] += ratings[i]
#         recommendations.set_value(row, 'rating', row['rating'] + ratings[i])
    i += 1

In [26]:
recommendations.sort_values('rating' , ascending = False)['movie'][:15]

214                                     The Intern
112                             Dallas Buyers Club
26                                           50/50
30                                     Bridesmaids
29                                   Green Lantern
4                                   X: First Class
25                           Friends with Benefits
9      Pirates of the Caribbean: On Stranger Tides
10                            Crazy, Stupid, Love.
18                  Transformers: Dark of the Moon
171                                  The Interview
226                                     Steve Jobs
134                                    Warm Bodies
2                                             Thor
71                     Snow White and the Huntsman
Name: movie, dtype: object